Simplest approach using only BoW to preprocess and then using my 2 personal favorite ML models - Random Forest and XGBoost 

In [33]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.shape

(404290, 6)

In [15]:
new_df = df.sample(10000)

In [16]:
new_df.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [17]:
ques_df = new_df[['question1','question2']]
ques_df.head()

,question1,question2
99041,How frequently can I have chicken livers safely?,Is it safe for dogs to eat chicken? Why or why...
353781,How do I see my older Instagram Direct Message...,If I direct message someone on Instagram and t...
131822,Should you be afraid of death?,Who is afraid of death?
336806,How should I treat a torn off toenail?,How do you treat a bruise under a toenail?
389738,What ROI will Blume Ventures get by investing ...,What is OSP Drafting?


In [18]:
from sklearn.feature_extraction.text import CountVectorizer

# questions = list(ques_df['question1']) + list(ques_df['question2'])

cv = CountVectorizer(max_features=3000)
# X = cv.fit_transform(questions).astype(np.float32).toarray()


In [19]:
q1_arr = cv.fit_transform(ques_df['question1']).toarray()
q2_arr = cv.transform(ques_df['question2']).toarray()  # Using transform, not fit_transform

In [20]:
temp_df1 = pd.DataFrame(q1_arr, index= ques_df.index)
temp_df2 = pd.DataFrame(q2_arr, index= ques_df.index)
temp_df = pd.concat([temp_df1, temp_df2], axis=1)
temp_df.shape

(10000, 6000)

In [21]:
temp_df

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
99041,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
353781,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
131822,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
336806,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
389738,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
386682,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
125553,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,0,0,0,0,0
268718,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
61410,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
temp_df['is_duplicate'] = new_df['is_duplicate']

In [23]:
temp_df.head()

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,is_duplicate
99041,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
353781,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
131822,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
336806,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
389738,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
print(temp_df.columns)

Index([             0,              1,              2,              3,
                    4,              5,              6,              7,
                    8,              9,
       ...
                 2991,           2992,           2993,           2994,
                 2995,           2996,           2997,           2998,
                 2999, 'is_duplicate'],
      dtype='object', length=6001)


In [25]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(temp_df.iloc[:,0:-1].values,temp_df.iloc[:,-1].values,test_size=0.2,random_state=1)

In [30]:
print(X_train.shape)
print(y_train.shape)

(8000, 6000)
(8000,)


In [35]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)

accuracy_score(y_test,y_pred)

0.731

In [34]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train,y_train)
y_pred = xgb.predict(X_test)

accuracy_score(y_test,y_pred)

0.7185